**Add information we care about to the geo json**

In [1]:
#import dependancies
import pandas as pd
import geojson

In [2]:
#set years that we have data for
years = [2016,2020]
#loop through the years
for year in years:
    #open the geojson
    geofile = f"./Data/Geojson/ConDistricts{year}.geojson"
    #make sure file is closed at end to save memory
    with open(geofile,'r') as f:
        #load geojson
        geodata = geojson.load(f)
        #open state and district data
        elecbydist = pd.read_csv("./Data/election_results_by_district.csv")
        elecbystate = pd.read_csv("./Data/election_results_by_state.csv")
        #loop through features of geojson to make changes
        for feat in geodata['features']:
            proptoadd = {}#place holder for dictionaries to add to the feature
            #get information from district to add to geojson
            samedist = elecbydist.loc[(elecbydist["Year"] == year)&(elecbydist["state_fips"]==int(feat.properties['STATEFP']))&(elecbydist["Congressional_District"]==feat.properties['NAMELSAD'])]
            #account for misssing data
            if samedist.empty:
                proptoadd["WINNER"] = "Unknown"
                proptoadd["DEMVOTES"] = 0
                proptoadd["REPVOTES"] = 0
            else:
                #check winner of election
                if (samedist["District_dem_votes"].item() > samedist["District_rep_votes"].item()):
                    #add democrat winner information
                    proptoadd["WINNER"] = "Democrat"
                    proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                    proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()
                else:
                    # add republican winner information
                    proptoadd["WINNER"] = "Republican"
                    proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                    proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()
            #Edit geojson with state election information
            samestate = elecbystate.loc[(elecbystate["Year"] == year)&(elecbystate["state_fips"]==int(feat.properties['STATEFP']))]
            #account for missing data
            if samestate.empty:
                proptoadd["STATENAME"] = "N/A"
                proptoadd["STATEWINNER"] = "Unknown"
                proptoadd["STATEDEMVOTES"] = 0
                proptoadd["STATEREPVOTES"] = 0
            else:
                if (samestate["State_dem_votes"].item() > samestate["State_rep_votes"].item()):
                    #add infor for democrat winner
                    proptoadd["STATENAME"] = samestate["State"].item()
                    proptoadd["STATEWINNER"] = "Democrat"
                    proptoadd["STATEDEMVOTES"] = samestate["State_dem_votes"].item()
                    proptoadd["STATEREPVOTES"] = samestate["State_rep_votes"].item()
                else:
                    #add info for republican winner
                    proptoadd["STATENAME"] = samestate["State"].item()
                    proptoadd["STATEWINNER"] = "Republican"
                    proptoadd["STATEDEMVOTES"] = samestate["State_dem_votes"].item()
                    proptoadd["STATEREPVOTES"] = samestate["State_rep_votes"].item()          
            #add information to geojson
            feat.properties.update(proptoadd)
        #open geojson for writing
        with open(f"./Data/Geojson/RevConDistricts{year}.geojson",'w') as out:
            #write the data
            geojson.dump(geodata,out)
        